In [30]:
pip install indic-nlp-library


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import re
try:
  from indicnlp.tokenize import indic_tokenize
except ImportError:
  print("indic-nlp library not found. Skipping tokenization.")


In [32]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [33]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
train_df = pd.read_csv("/content/drive/My Drive/Data/hindi_train.csv")
val_df = pd.read_csv("/content/drive/My Drive/Data/hindi_val.csv")
test_df = pd.read_csv("/content/drive/My Drive/Data/hindi_test.csv")

In [36]:
print("Train Dataset Sample:")
print(train_df.head())

print("\nValidation Dataset Sample:")
print(val_df.head())

print("\nTest Dataset Sample:")
print(test_df.head())

Train Dataset Sample:
   label                                       text
0      0              भीड़ में  बहुत  हब्सी मिलेंगे
1      0  साले बेवकूफ अपनी मां मक्खियां  तो हटा दें
2      0           बुर देदो तो मुह में लंड ले लो तो
3      0       कुत्ता वहा है चिल्ला तू क्यों रहा है
4      1  चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok

Validation Dataset Sample:
   label                                               text
0      0                          Comment box चालू कर झवाडे
1      0  आपके पास पुख्ता सबूत है, क्या या फिर कुछ भी। उ...
2      1  👬दोस्ती होती है – One Time 🙃 हम निभाते है – So...
3      1  चाय लवर डाॅ. को छोड़ देगे or mo. कोई छोड़ नहीं...
4      1  जो अनपढ़ लोग दिन-रात हिंदू मुस्लिम करते रहते ह...

Test Dataset Sample:
   label                                               text
0      0  मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सो...
1      1        और दिवाली में भी पूरा देश पड़ाका नहीं फोडात
2      1      कुत्ता बिल्ली पाल लेना मगर गलत फहमी कभी नहीं।
3      0      ते

In [37]:
print("Train Dataset Columns:")
print(train_df.columns)

print("\nValidation Dataset Columns:")
print(val_df.columns)

print("\nTest Dataset Columns:")
print(test_df.columns)


Train Dataset Columns:
Index(['label', 'text'], dtype='object')

Validation Dataset Columns:
Index(['label', 'text'], dtype='object')

Test Dataset Columns:
Index(['label', 'text'], dtype='object')


In [38]:
stopwords_file = '/content/drive/My Drive/Data/final_stopwords.txt'  # Replace with the path to your stopwords file
with open(stopwords_file, 'r', encoding='utf-8') as file:
    stop_words_list = [line.strip() for line in file]
stop_words_hindi = set(stop_words_list)

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words_hindi])

train_df['text'] = train_df['text'].apply(remove_stopwords)
train_df['tokens'] = train_df['text'].apply(indic_tokenize.trivial_tokenize)

In [39]:
def preprocess_text(text):
  text = text.lower()  # Lowercase
  text = re.sub(r'[^\w\s\u0900-\u097F]', ' ', text)  # Keeps Hindi characters, whitespace, and alphanumeric
  text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
  # Apply stopword removal if stopwords_file is defined
  if stopwords_file:
      text = remove_stopwords(text)
  # Tokenization (using indic_nlp if available)
  try:
      text = indic_tokenize.trivial_tokenize(text)
      text = ' '.join(text)  # Join tokens back into string
  except ImportError:
      print("indic_nlp library not found. Skipping tokenization.")
  return text

In [40]:
print("Preprocessing Training Data:")
train_df["text_preprocessed"] = train_df["text"].apply(preprocess_text)
print("Sample Preprocessed Training Text:")
print(train_df["text_preprocessed"].head())

Preprocessing Training Data:
Sample Preprocessed Training Text:
0                  भीड़ हब्सी मिलेंगे
1    साले बेवकूफ मां मक्खियां हटा दें
2              बुर देदो मुह लंड ले लो
3                कुत्ता वहा चिल्ला तू
4           चाय नहीं पीता हूं छोड़ ok
Name: text_preprocessed, dtype: object


In [41]:
print("Preprocessing Validation Data:")
val_df["text_preprocessed"] = val_df["text"].apply(preprocess_text)
print("Sample Preprocessed Validation Text:")
print(val_df["text_preprocessed"].head())

Preprocessing Validation Data:
Sample Preprocessed Validation Text:
0                               comment box चालू झवाडे
1    आपके पास पुख्ता सबूत भी । आधार कार्ड के कारण व...
2    दोस्ती one time निभाते some time याद करो any t...
3    चाय लवर डाॅ छोड़ देगे or mo छोड़ नहीं ओर m bhi...
4    अनपढ़ लोग दिन रात हिंदू मुस्लिम रहते सोचना देख...
Name: text_preprocessed, dtype: object


In [42]:
print("Preprocessing Test Data:")
test_df["text_preprocessed"] = test_df["text"].apply(preprocess_text)
print("Sample Preprocessed Test Text:")
print(test_df["text_preprocessed"].head())

Preprocessing Test Data:
Sample Preprocessed Test Text:
0                       नही सोच निकले सोच फंसा
1                 दिवाली देश पड़ाका नहीं फोडात
2       कुत्ता बिल्ली पाल लेना गलत फहमी नहीं ।
3    तेरी गांड प्याज काट देगा गुज्जर भोसड़ी के
4                  बंगाली साड़ी नहीं पहना दीदी
Name: text_preprocessed, dtype: object


In [43]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Dataset class
class ToxicCommentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



In [45]:
# Creating Datasets and Dataloaders
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = ToxicCommentDataset(
        texts=df["text_preprocessed"].tolist(),
        labels=df["label"].tolist(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

BATCH_SIZE = 16
MAX_LEN = 128

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)


In [46]:
# Training and Evaluation Functions
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels.extend(batch['labels'].numpy())
            outputs = model(input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
    return accuracy_score(labels, preds)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [47]:
# Training Loop
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_data_loader, optimizer, device)
    val_accuracy = evaluate(model, val_data_loader, device)
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print(f'Train loss: {train_loss}')
    print(f'Validation accuracy: {val_accuracy}')


Epoch 1/5
Train loss: 0.4767965018194281
Validation accuracy: 0.8317479191438764
Epoch 2/5
Train loss: 0.35736179521835737
Validation accuracy: 0.8315992865636147
Epoch 3/5
Train loss: 0.28990627370520883
Validation accuracy: 0.8434898929845422
Epoch 4/5
Train loss: 0.230724264415964
Validation accuracy: 0.8347205707491082
Epoch 5/5
Train loss: 0.17441970582242625
Validation accuracy: 0.832936979785969


In [48]:
# Evaluate on Test Set
test_accuracy = evaluate(model, test_data_loader, device)
print(f'Test accuracy: {test_accuracy}')


Test accuracy: 0.8376932223543401


In [49]:
# Function to get predictions for a sample of data
def get_predictions(model, data_loader, device):
    model.eval()
    texts = []
    preds = []
    labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels.extend(batch['labels'].cpu().numpy())
            outputs = model(input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            # Decode input_ids to get the original text
            texts.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids])
    return texts, preds, labels

# Get predictions for the test set
test_texts, test_preds, test_labels = get_predictions(model, test_data_loader, device)

# Display a sample of the predictions
sample_size = 10  # Number of samples to display
print(f'Displaying {sample_size} samples from the test set predictions:\n')

for i in range(sample_size):
    print(f'Text: {test_texts[i]}')
    print(f'Actual Label: {test_labels[i]}')
    print(f'Predicted Label: {test_preds[i]}')
    print('---')


Displaying 10 samples from the test set predictions:

Text: प्यारी लगरही तुम्हे प्यार सनम आय लव्ह यु संचिता
Actual Label: 1
Predicted Label: 1
---
Text: अरे पागल औरत हर विडियो एकटिगं करेगी क अलग ले
Actual Label: 0
Predicted Label: 0
---
Text: दोस्तों दीपावली हार्दिक शुभकामनाएं ईश्वर प्रार्थना वो आपको हमेशा स्वस्थ ऊर्जावान
Actual Label: 1
Predicted Label: 1
---
Text: लंगर mae बोटी नहीं मिलती chutiye
Actual Label: 0
Predicted Label: 0
---
Text: भाई लोग ध्यान देखो 2 घोड़े गधा
Actual Label: 0
Predicted Label: 0
---
Text: इन । पुलिस वालो कुत्तों इतनी हिम्मत समझ लो कुत्ते नौकरी हाथ धो बैठे कुत्ते जवान सिर्फ मास्क नहीं लगाने इतनी बुरी पिटाई
Actual Label: 0
Predicted Label: 0
---
Text: लोग दबंग जी तुझे लगता छोटी छोटी चीजों दिखा लोग पागल देगी इश छोटी सी चीज 100 रुपया खेल आ
Actual Label: 0
Predicted Label: 0
---
Text: जय राम देव बाबा
Actual Label: 1
Predicted Label: 1
---
Text: ना शरमाओ लाजवाब सुन्दर
Actual Label: 1
Predicted Label: 1
---
Text: बीजेपी विश्वास करके दिखाया बाकी 70 साल चुटिया बनाया